In [2]:
%logstop
%logstart -rtq ~/.logs/ml.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [5]:
import numpy as np
import pandas as pd

In [6]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [8]:
data.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


In [9]:
data.shape

(13892, 29)

In [10]:
data['STATE'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'GU'], dtype=object)

In [11]:
# Predict amount of fine based on STATE of nursing home

In [12]:
fine_counts.groupby(data['STATE']).mean().head()

STATE
AK    0.812500
AL    0.233010
AR    0.626866
AZ    0.079365
CA    0.289179
Name: FINE_CNT, dtype: float64

In [13]:
fine_totals.groupby(data['STATE']).mean().head() # groups all entries for each STATE and outputs their mean
# pandas documentation good example!
# here, use data['STATE'] as a kind of index (not actually in DF y, like normal groupby)

STATE
AK    15932.750000
AL    13672.320388
AR    17596.681592
AZ     1512.722222
CA     8054.977612
Name: FINE_TOT, dtype: float64

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

For general template, look up custom estimator in sklearn documentation!

In [14]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

# want to predict method to output mean of state!
class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    '''Assume: X is a pd DF, y is a pd Series.
       Grouper is the name of a column in X.
       Goal: (1) (fit) group the data in y, using the column 'grouper' from X (indexes) and compute mean of y (in groups),
       store in self.group_averages (dictionary)
       (2) (predict) will use the column 'grouper' in X to predict values
    '''
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {} # fill this dict in the fit method, don't NEED to do this here, but nice reminder
        self.global_average = 0 # reminder
        
    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        # if input not pandas DF, convert to one
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict() # .to_dict() converts series to dict, as required by __init__ method. Easy to look things up!
        self.global_average = y.mean()
        return self # A reference to the object that is residing in memory. 
                    # Line of output represents return self. It allows method chaining.

    # Should ALWAYS return one row for each row in X (1 ouput per observation i in [0, N])
    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
        # if input not pandas DF, convert to one
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        # use .get() method to predict global mean when looking up State not in dict group_averages
        return [self.group_averages.get(group, self.global_average) for group in X[self.grouper]] # grouper can be list of *multiple* columns, in general

In [15]:
d = {'AL':10, 'AK':14}

In [16]:
d['AK'] # can use square brackets, but returns key error if key does not exist

14

In [17]:
print(d.get('AS')) # .get() method for dicts does not return key error! Returns 'None'

None


In [18]:
d.get('AS', 1200) # can supply output value for keys that do not exist in dict!

1200

In [19]:
gme = GroupMeanEstimator(grouper='STATE') # could input any 'grouper' column that the mean is desired for
gme.fit(data, fine_totals)

GroupMeanEstimator(grouper='STATE')

In [20]:
gme.group_averages # attribute stored in fit method

{'AK': 15932.75,
 'AL': 13672.320388349515,
 'AR': 17596.6815920398,
 'AZ': 1512.7222222222222,
 'CA': 8054.977611940299,
 'CO': 22112.54591836735,
 'CT': 8438.121359223302,
 'DC': 27333.933333333334,
 'DE': 24899.0,
 'FL': 16612.338211382114,
 'GA': 29459.975,
 'GU': 0.0,
 'HI': 16133.309523809523,
 'IA': 16565.303571428572,
 'ID': 42741.942028985504,
 'IL': 6634.197226502311,
 'IN': 5626.954545454545,
 'KS': 24420.79109589041,
 'KY': 32656.315384615384,
 'LA': 5611.819277108434,
 'MA': 16722.25925925926,
 'MD': 42806.67661691542,
 'ME': 1275.5869565217392,
 'MI': 21437.39746835443,
 'MN': 3219.326530612245,
 'MO': 7635.160997732426,
 'MS': 7595.4812834224595,
 'MT': 32754.970588235294,
 'NC': 30445.040506329115,
 'ND': 560.171052631579,
 'NE': 8377.755319148937,
 'NH': 260.45588235294116,
 'NJ': 3490.756838905775,
 'NM': 39652.64705882353,
 'NV': 3050.1964285714284,
 'NY': 2213.51526032316,
 'OH': 8214.822977725675,
 'OK': 11812.111111111111,
 'OR': 12365.983606557376,
 'PA': 9216.96

In [21]:
gme.predict(data) # different values represent nursing homes in different states!

[13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [22]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [23]:
state_model.predict(data.sample(5))

[18425.87278106509,
 16565.303571428572,
 18425.87278106509,
 8214.822977725675,
 7635.160997732426]

However, what if we have data from a nursing home in a state (or territory) of the US which is **not** in the training data?

In [24]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}])) # trying to look up key that isn't in dictionary!
# need to add to fit method to inpute GLOBAL mean

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [25]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Only using numeric and boolean (0/1) features.

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [26]:
from sklearn.impute import SimpleImputer

In [27]:

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

# converts to dataframe and then selects the columns we want
class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('impute', SimpleImputer())
])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [28]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any() # SimpleImputer outputs nparray

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [29]:
# need classification algorithm
from sklearn.linear_model import LogisticRegression

In [30]:
simple_features_model = Pipeline([
    ('simple', simple_features),
    # add your estimator here
    ('est', LogisticRegression())
])

In [31]:
simple_features_model.fit(data, fine_counts > 0) # fine_counts > 0 is our label!

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('simple',
                 Pipeline(memory=None,
                          steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('impute',
                                  SimpleImputer(add_indicator=False, copy=True,
                        

In [32]:
simple_features_model.score(data, fine_counts > 0)  # accuracy of our classifier (c=0.5)

0.6939965447739707

In [33]:
simple_features_model.predict_proba(data)[:5] # prob of classification!

array([[0.80225756, 0.19774244],
       [0.85556825, 0.14443175],
       [0.69513108, 0.30486892],
       [0.71375592, 0.28624408],
       [0.68032557, 0.31967443]])

In [34]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score:  1.002750512350338


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [35]:
data['OWNERSHIP'].unique()

array(['For profit - Corporation', 'Government - County',
       'Non profit - Other', 'Non profit - Corporation',
       'For profit - Individual', 'For profit - Partnership',
       'Government - City', 'Non profit - Church related',
       'Government - City/county', 'Government - Federal',
       'For profit - Limited Liability company', 'Government - State',
       'Government - Hospital district'], dtype=object)

In [36]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder())
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe', OneHotEncoder())
])

# could have used ColumnTransformer to specify these two columns together

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [37]:
categorical_features.fit_transform(data)[:2].toarray()

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [38]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data).toarray()).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    # add your estimator here
    ('estimator', RandomForestClassifier(random_state=42))
     # or can use GridSearch but this gives worse score here...
     #GridSearchCV(
     #   RandomForestClassifier(), 
     #   param_grid = {'max_depth': range(10, 60, 10)}, cv=5, verbose=1))
])

In [40]:
categorical_features_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories=None,
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='error',
   

In [41]:
# look at optimal `max_depth` hyperparameter if using GridCV:

#categorical_features_model.named_steps['estimator'].best_params_

In [42]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  0.9812068143912321


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [43]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In general, with $n$ features, we'd have $1+2n+\frac{n(n-1)}{2}$ features in the new feature matrix (with the default settings in `PolynomalFeatures`).

In [44]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD

business_model = Pipeline([
    ('features', business_features),
    # add your estimator here
    ('poly', PolynomialFeatures(degree=2)), # create new poly features
    ('svd', TruncatedSVD(n_components=30)), # reduce dim back to 30 (PCA alternative)
    ('estimator', LogisticRegression())
])

In [45]:
business_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('simple',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                      

In [46]:
# percentage of variance explained by each of the 30 features
business_model.named_steps['svd'].explained_variance_ratio_

array([9.12524525e-01, 8.61830020e-02, 1.21773328e-03, 5.57836243e-05,
       6.36927534e-06, 3.42925958e-06, 1.72968594e-06, 1.66021214e-06,
       9.13666437e-07, 8.82922852e-07, 6.46486577e-07, 5.58422744e-07,
       4.47920592e-07, 4.03815175e-07, 3.22942317e-07, 2.73621364e-07,
       2.35252872e-07, 1.74717046e-07, 1.57560676e-07, 1.29537327e-07,
       1.23503678e-07, 9.01382555e-08, 8.50076512e-08, 5.71031036e-08,
       5.97985784e-08, 4.52006357e-08, 3.68239963e-08, 1.92901424e-08,
       1.62372256e-08, 1.30374224e-08])

In [47]:
business_model.named_steps['svd'].explained_variance_ratio_.cumsum()
# only need first 3 features to explain 99.99% of variance!

array([0.91252453, 0.99870753, 0.99992526, 0.99998104, 0.99998741,
       0.99999084, 0.99999257, 0.99999423, 0.99999515, 0.99999603,
       0.99999668, 0.99999723, 0.99999768, 0.99999809, 0.99999841,
       0.99999868, 0.99999892, 0.99999909, 0.99999925, 0.99999938,
       0.9999995 , 0.99999959, 0.99999968, 0.99999974, 0.99999979,
       0.99999984, 0.99999988, 0.9999999 , 0.99999991, 0.99999993])

In [48]:
grader.score.ml__business_model(positive_probability(business_model))

Error!
You have been rate limited for exceeding the limit of 3 per 1 minute.
Please slow down your submission rate.


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to **predict the cycle 2 total score.** Already popped out these labels from OG dataset.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [49]:
data.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


In [52]:
data['CYCLE_1_SURVEY_DATE'].head()

0    2017-04-06
1    2017-03-16
2    2016-10-20
3    2017-03-09
4    2017-06-01
Name: CYCLE_1_SURVEY_DATE, dtype: object

In [55]:
# convert to datetime data type, as pd can do calcs in this form!
pd.to_datetime(data['CYCLE_1_SURVEY_DATE'].head())

# values converts to nanoseconds; then convert to float number
(pd.to_datetime(data['CYCLE_1_SURVEY_DATE']) - pd.to_datetime(data['CYCLE_2_SURVEY_DATE'])).values.astype(float)

array([2.72160e+16, 3.50784e+16, 2.54880e+16, ..., 2.48832e+16,
       2.85984e+16, 8.46720e+15])

In [60]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        diff = (pd.to_datetime(X[self.t2_col]) - pd.to_datetime(X[self.t1_col])).values.astype(float).reshape(-1,1)
        return diff
    

In [61]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In [62]:
time_feature.fit_transform(data)[:5]

array([[-2.72160e+16],
       [-3.50784e+16],
       [-2.54880e+16],
       [-3.38688e+16],
       [-3.32640e+16]])

In the cell below we'll collect the cycle 1 survey features.

In [63]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [82]:
from sklearn.ensemble import RandomForestRegressor

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add PolynomialFeatures?? add TruncatedSVD?
    ('poly', PolynomialFeatures(degree=2)),
    ('svd', TruncatedSVD(n_components=30)),
    
    # add your estimator here
    ('rfr', GridSearchCV(
        RandomForestRegressor(n_estimators=200, n_jobs=2, random_state=0),
        param_grid = {'min_samples_leaf': range(20, 35, 5)},
        cv=5, verbose=2))
])

# 8.5 mins without middle section
# 12.2 mins with (better!)

In [83]:
survey_model.fit(data, cycle_2_score.astype(int))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] min_samples_leaf=20 .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .............................. min_samples_leaf=20, total=  53.7s
[CV] min_samples_leaf=20 .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   53.7s remaining:    0.0s


[CV] .............................. min_samples_leaf=20, total=  52.3s
[CV] min_samples_leaf=20 .............................................
[CV] .............................. min_samples_leaf=20, total=  51.9s
[CV] min_samples_leaf=20 .............................................
[CV] .............................. min_samples_leaf=20, total=  51.4s
[CV] min_samples_leaf=20 .............................................
[CV] .............................. min_samples_leaf=20, total=  51.6s
[CV] min_samples_leaf=25 .............................................
[CV] .............................. min_samples_leaf=25, total=  48.7s
[CV] min_samples_leaf=25 .............................................
[CV] .............................. min_samples_leaf=25, total=  48.2s
[CV] min_samples_leaf=25 .............................................
[CV] .............................. min_samples_leaf=25, total=  49.1s
[CV] min_samples_leaf=25 .............................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 12.2min finished


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('business',
                                                 FeatureUnion(n_jobs=None,
                                                              transformer_list=[('simple',
                                                                                 Pipeline(memory=None,
                                                                                          steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                               

In [84]:
survey_model.score(data, cycle_2_score) # R^2 score - low as not overfit to training data, but generalises well!

0.12399951409068456

In [85]:
survey_model.predict(data)[:5] # predict on training set

array([53.57452872, 48.07313383, 56.96184166, 43.08228783, 45.40424706])

In [86]:
grader.score.ml__survey_model(survey_model.predict)

Your score:  1.104289262709242


- Truncated SVD for sparse matrices
- PCA for dense matrices (due to calculations)